<a href="https://colab.research.google.com/github/AshleyW09/NLP_Capstone/blob/main/04_preprocessing_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch]
! pip install transformers
! pip install datasets
! pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
#import tensorflow as tf
from transformers import TrainingArguments, Trainer

In [3]:
from google.colab import files
uploaded = files.upload()

Saving reviews_data.csv to reviews_data.csv


In [4]:
import io
reviews_data = pd.read_csv(io.BytesIO(uploaded['reviews_data.csv']))

In [5]:
reviews_data.rename(columns = {'Rating':'labels'}, inplace = True)

In [6]:
train, validate, test = np.split(reviews_data.sample(frac=1, random_state=42), [int(.6*len(reviews_data)), int(.8*len(reviews_data))])

In [7]:
train = Dataset.from_pandas(train)
validate = Dataset.from_pandas(validate)
test = Dataset.from_pandas(test)

In [8]:
train = train.class_encode_column("labels")
validate = validate.class_encode_column("labels")
test = test.class_encode_column("labels")

Stringifying the column:   0%|          | 0/13584 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/13584 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/4528 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4528 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/4529 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4529 [00:00<?, ? examples/s]

In [9]:
dataset = DatasetDict()

In [10]:
dataset['train'] = train
dataset['validate'] = validate
dataset['test'] = test

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Review Text', 'labels', '__index_level_0__'],
        num_rows: 13584
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'Review Text', 'labels', '__index_level_0__'],
        num_rows: 4528
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Review Text', 'labels', '__index_level_0__'],
        num_rows: 4529
    })
})

In [12]:
dataset['train'].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'Review Text': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['1', '2', '3', '4', '5'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [13]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
def tokenization(sentence):
    return tokenizer(sentence["Review Text"], truncation=True)

tokenized_dataset = dataset.map(tokenization, batched=True)

Map:   0%|          | 0/13584 [00:00<?, ? examples/s]

Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

In [16]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Review Text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 13584
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'Review Text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 4528
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Review Text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [27]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [23]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["validate"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,)


In [24]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.825200,0.813128,0.666961,0.640551
2,0.686200,0.791866,0.668507,0.663547
3,0.483800,0.921920,0.666961,0.664188


<ipython-input-22-b1bb99dfba71>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=5094, training_loss=0.6911293811595903, metrics={'train_runtime': 647.709, 'train_samples_per_second': 62.917, 'train_steps_per_second': 7.865, 'total_flos': 1301613570438000.0, 'train_loss': 0.6911293811595903, 'epoch': 3.0})

In [25]:
trainer.evaluate()

{'eval_loss': 0.9219204783439636,
 'eval_accuracy': 0.6669611307420494,
 'eval_f1': 0.6641877690971729,
 'eval_runtime': 20.7557,
 'eval_samples_per_second': 218.157,
 'eval_steps_per_second': 27.27,
 'epoch': 3.0}